<a href="https://colab.research.google.com/github/Madhuram2901/AcWOC-MovieMusic/blob/main/MovMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
from difflib import get_close_matches
from typing import List, Dict, Union


class GenreRecommendationSystem:
    def __init__(self, movies_path: str, music_path: str):
        self.movies_df = self._load_movie_data(movies_path)
        self.music_genres_df = self._load_music_data(music_path)
        self.genre_mapping = self._create_genre_mapping()

    def _load_movie_data(self, path: str) -> pd.DataFrame:
        """Loads and processes the movies dataset."""
        try:
            movies_df = pd.read_csv(path, usecols=['id', 'genres', 'title'])
            movies_df['genres'] = movies_df['genres'].apply(
                lambda x: [genre['name'] for genre in json.loads(x)]
            )
            return movies_df
        except Exception as e:
            raise ValueError(f"Error loading movie data: {e}")

    def _load_music_data(self, path: str) -> pd.DataFrame:
        """Loads the music genres dataset."""
        try:
            music_df = pd.read_csv(path)

            # Ensure necessary columns are present
            required_columns = ['genres', 'track_names']
            missing_columns = [col for col in required_columns if col not in music_df.columns]
            if missing_columns:
                raise ValueError(f"Missing columns in music data: {missing_columns}")

            return music_df
        except Exception as e:
            raise ValueError(f"Error loading music data: {e}")

    def _create_genre_mapping(self) -> Dict[str, List[str]]:
        """Creates a mapping of movie genres to music genres."""
        return {
            'Action': ['electronic', 'rock', 'epic', 'bass music', 'drum and bass'],
            'Adventure': ['world', 'cinematic', 'orchestral', 'folk'],
            'Fantasy': ['21st century classical', 'orchestral', 'cinematic', 'world'],
            'Science Fiction': ['electronic', 'ambient', 'abstract', 'abstract beats'],
            'Crime': ['dark ambient', 'abstract hip hop', 'trip hop'],
            'Drama': ['classical', 'acoustic', 'piano', 'ambient'],
            'Comedy': ['a cappella', 'pop', 'quirky', 'fun'],
            'Romance': ['acoustic', 'piano', 'vocal'],
            'Horror': ['dark ambient', 'abstract', 'atmospheric'],
            'Thriller': ['dark ambient', 'electronic', 'atmospheric'],
            'Animation': ['8-bit', 'fun', 'quirky', 'orchestral'],
            'Family': ['acoustic', 'pop', 'folk', 'fun']
        }

    def find_movie(self, movie_title: str) -> Union[None, pd.Series]:
        """Finds a movie by title."""
        matches = self.movies_df[self.movies_df['title'].str.contains(movie_title, case=False, na=False)]
        return matches.iloc[0] if not matches.empty else None

    def get_related_music_genres(self, movie_genres: List[str]) -> List[str]:
        """Finds related music genres for given movie genres."""
        related_music_genres = set()
        for genre in movie_genres:
            if genre in self.genre_mapping:
                related_music_genres.update(self.genre_mapping[genre])
        return list(related_music_genres)

    def recommend_music_based_on_movie(self, movie_title: str, num_recommendations: int = 5) -> Union[List[Dict[str, str]], str]:
        """Recommends music tracks based on a movie title."""
        movie = self.find_movie(movie_title)
        if movie is None:
            return f"Movie not found. Did you mean one of these: {', '.join(get_close_matches(movie_title, self.movies_df['title'].tolist(), n=3))}?"

        movie_genres = movie['genres']
        related_music_genres = self.get_related_music_genres(movie_genres)

        if not related_music_genres:
            return "No matching music genres found for this movie's genres."

        matching_music = self.music_genres_df[
            self.music_genres_df['genres'].str.lower().isin([g.lower() for g in related_music_genres])
        ]

        if matching_music.empty:
            return "No music recommendations found for the related genres."

        recommendations = matching_music.sample(n=min(num_recommendations, len(matching_music)))

        return recommendations[['genres', 'track_names']].to_dict('records')


def main():
    print("Welcome to the Movie-to-Music Recommendation System!")
    recommender = GenreRecommendationSystem(
        movies_path='tmdb_5000_movies.csv',
        music_path='extended_data_by_genres.csv'
    )

    while True:
        movie_title = input("\nEnter a movie title (or 'quit' to exit): ")
        if movie_title.lower() == 'quit':
            print("Goodbye!")
            break

        recommendations = recommender.recommend_music_based_on_movie(movie_title)
        if isinstance(recommendations, str):
            print(recommendations)
        else:
            print(f"\nRecommended music tracks for '{movie_title}':")
            for i, rec in enumerate(recommendations, 1):
                print(f"{i}. Genre: {rec['genres']} - Track: {rec['track_names']}")


if __name__ == "__main__":
    main()

Welcome to the Movie-to-Music Recommendation System!

Enter a movie title (or 'quit' to exit): Avatar

Recommended music tracks for 'Avatar':
1. Genre: 21st century classical - Track: Whispers of 21St Century Classical
2. Genre: drum and bass - Track: Echoes of Drum And Bass
3. Genre: folk - Track: Rhythms of Folk
4. Genre: world - Track: Dreams of World
5. Genre: bass music - Track: Whispers of Bass Music
